In [2]:
import pandas as pd
from WindPy import *

def ema(data, n=12, val_name="close"):
    import numpy as np
    '''
        指数平均数指标 Exponential Moving Average
        Parameters
        ------
          data:pandas.DataFrame
                      通过 get_h_data 取得的数据
          n:int
                      移动平均线时长，时间单位根据data决定
          val_name:string
                      计算哪一列的列名，默认为 close 
        return
        -------
          EMA:numpy.ndarray<numpy.float64>
              指数平均数指标
    '''

    prices = []

    EMA = []

    for index, row in data.iterrows():
        if index == 0:
            past_ema = row[val_name]
            EMA.append(row[val_name])
        else:
            # Y=[2*X+(N-1)*Y’]/(N+1)
            today_ema = (2 * row[val_name] + (n - 1) * past_ema) / (n + 1)
            past_ema = today_ema

            EMA.append(today_ema)

    return np.asarray(EMA)


def macd(data, quick_n=12, slow_n=26, dem_n=9, val_name="close"):
    import numpy as np
    '''
        指数平滑异同平均线(MACD: Moving Average Convergence Divergence)
        Parameters
        ------
          data:pandas.DataFrame
                      通过 get_h_data 取得的
          quick_n:int
                      DIFF差离值中快速移动天数
          slow_n:int
                      DIFF差离值中慢速移动天数
          dem_n:int
                      DEM讯号线的移动天数
          val_name:string
                      计算哪一列的列名，默认为 close 
        return
        -------
          OSC:numpy.ndarray<numpy.float64>
              MACD bar / OSC 差值柱形图 DIFF - DEM
          DIFF:numpy.ndarray<numpy.float64>
              差离值
          DEM:numpy.ndarray<numpy.float64>
              讯号线
    '''

    ema_quick = np.asarray(ema(data, quick_n, val_name))
    ema_slow = np.asarray(ema(data, slow_n, val_name))
    DIFF = ema_quick - ema_slow
    data["diff"] = DIFF
    DEM = ema(data, dem_n, "diff")
    BAR = (DIFF - DEM)*2
    data['dem'] = DEM
    data['bar'] = BAR
    return  DIFF, DEM, BAR

    

In [3]:
df = pd.read_csv('./601127.csv')

df.columns
df.rename(columns={df.columns[0]:"date"}, inplace=True)
df

,date,close
0,2019-01-02 10:00:00,16.66
1,2019-01-02 11:00:00,16.65
2,2019-01-02 12:00:00,16.62
3,2019-01-02 14:00:00,16.70
4,2019-01-02 15:00:00,16.70
5,2019-01-03 10:00:00,16.60
6,2019-01-03 11:00:00,16.13
7,2019-01-03 12:00:00,16.15
8,2019-01-03 14:00:00,16.04
9,2019-01-03 15:00:00,16.08


In [4]:
diff, dem ,bar = macd(df)
df

,date,close,diff,dem,bar
0,2019-01-02 10:00:00,16.66,0.000000,0.000000,0.000000
1,2019-01-02 11:00:00,16.65,-0.000798,-0.000160,-0.001276
2,2019-01-02 12:00:00,16.62,-0.003807,-0.000889,-0.005836
3,2019-01-02 14:00:00,16.70,0.000261,-0.000659,0.001840
4,2019-01-02 15:00:00,16.70,0.003445,0.000162,0.006566
5,2019-01-03 10:00:00,16.60,-0.002077,-0.000286,-0.003582
6,2019-01-03 11:00:00,16.13,-0.043873,-0.009003,-0.069739
7,2019-01-03 12:00:00,16.15,-0.074523,-0.022107,-0.104832
8,2019-01-03 14:00:00,16.04,-0.106463,-0.038978,-0.134968
9,2019-01-03 15:00:00,16.08,-0.127082,-0.056599,-0.140966


In [6]:
df.to_csv("601127_macd.csv")

In [12]:

from tushare.util.formula import MACD
df2=pd.read_csv("./601127.csv")
result = MACD(df2['close'], 12,26,9)

result.to_csv("601127.tusharemacd.csv")
result  # 计算出的结果和上面是一样的，但是和同花顺有差别

,DIFF,DEA,MACD
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN
